In [1]:
import csv
import os
import cv2
import glob
import logging
import mediapipe as mp

import matplotlib.pyplot as plt

2024-03-10 12:40:19.173756: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-10 12:40:19.223156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
logging.basicConfig(filename='processing.log', level=logging.INFO, format='%(asctime)s - %(message)s')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

folder_path = 'datsaet_new'
data = []

video_count = 0

for video_file in glob.glob(os.path.join(folder_path, '*.avi')):
    video_count += 1
    logging.info(f'Processing video: {os.path.basename(video_file)}, Video {video_count} out of {len(glob.glob(os.path.join(folder_path, "*.avi")))}')

    video_capture = cv2.VideoCapture(video_file)
    
    keypoints_count = 0
    frames_count = 0

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                keypoints_count += 1
        frames_count += 1

    video_capture.release()
    
    video_name = os.path.basename(video_file)
    average_keypoints_per_frame = keypoints_count / frames_count if frames_count > 0 else 0

    data.append([video_name, keypoints_count, average_keypoints_per_frame])

csv_file = 'pose_keypoints_data.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Video Name', 'Total Keypoints', 'Average Keypoints per Frame'])
    writer.writerows(data)

print(f'Data saved to {csv_file}')


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
